<a href="https://colab.research.google.com/github/kaicong12/CNN-/blob/main/Cats%26Dogs_ValAcc%3D89_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/MyDrive/archive (4).zip"

In [ ]:
import keras
import os
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import cv2
from keras.callbacks import ReduceLROnPlateau

In [ ]:
train_dataset = []
classes = ['cats', 'dogs']
imgsize = 50
for cls_id, cls in enumerate(classes):
  path = '/content/dataset/training_set'
  path = os.path.join(path, cls)
  for img in os.listdir(path):
    img_data = cv2.imread(os.path.join(path, img))
    img_data = cv2.resize(img_data, (imgsize,imgsize))
    train_dataset.append([img_data, cls_id])


In [ ]:
datagen = ImageDataGenerator(
                                    rotation_range=45,
                                    shear_range=10,
                                    zoom_range=[1,1.3],
                                    horizontal_flip=True,
                                    vertical_flip=True,
)

In [ ]:
aug_data = []
for obj in train_dataset:
  obj[0] = obj[0].reshape((1,imgsize,imgsize,3))
  i = 1
  for d in datagen.flow(obj[0], batch_size=1):
    aug_data.append([d[0], obj[1]])
    i = i + 1
    if i == 15:
      break

In [ ]:
import random
random.shuffle(aug_data)

In [ ]:
x, y = [], []
for pix, cls in aug_data:
  x.append(pix)
  y.append(cls)

x = np.array(x)
x = x.astype('float32') / 255.0
y = np.array(y, dtype='uint8')

In [ ]:
x.shape

(112000, 50, 50, 3)

In [ ]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2)

In [ ]:
model = Sequential()
model.add(Conv2D(64*2, 3, activation='relu', input_shape=(imgsize,imgsize,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D())

model.add(Conv2D(64, 3, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D())

model.add(Flatten())
model.add(Dense(64*2*2, activation='relu'))
model.add(BatchNormalization())

model.add(Dense(64*2, activation='relu'))
model.add(BatchNormalization())

model.add(Dense(2, activation='softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

lr_reduction = ReduceLROnPlateau(patience=3, verbose=1, factor=0.25, min_lr=0.000003)

In [ ]:
model.fit(xtrain,ytrain,epochs=15,batch_size=64,validation_data=(xtest,ytest), callbacks=[lr_reduction])

Epoch 1/15
1400/1400 [==============================] - 29s 15ms/step - loss: 0.6177 - accuracy: 0.6710 - val_loss: 0.5762 - val_accuracy: 0.6943
Epoch 2/15
1400/1400 [==============================] - 20s 14ms/step - loss: 0.4569 - accuracy: 0.7819 - val_loss: 0.4869 - val_accuracy: 0.7705
Epoch 3/15
1400/1400 [==============================] - 20s 14ms/step - loss: 0.3568 - accuracy: 0.8426 - val_loss: 0.4391 - val_accuracy: 0.7987
Epoch 4/15
1400/1400 [==============================] - 20s 14ms/step - loss: 0.2425 - accuracy: 0.9000 - val_loss: 0.4767 - val_accuracy: 0.8085

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
Epoch 5/15
1400/1400 [==============================] - 20s 14ms/step - loss: 0.1135 - accuracy: 0.9611 - val_loss: 0.4581 - val_accuracy: 0.8407

Epoch 00005: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.
Epoch 6/15
1400/1400 [==============================] - 20s 14ms/step - loss: 0.0452 - accuracy: 0.9876 - val

In [ ]:
score = model.evaluate(xtest,ytest)

700/700 [==============================] - 3s 4ms/step - loss: 0.3748 - accuracy: 0.8833


In [ ]:
test = cv2.imread('/content/dog vs cat/dataset/test_set/dogs/dog.4001.jpg')
test = cv2.resize(test, (imgsize, imgsize))
test = test.reshape((1,imgsize,imgsize,3))
predict = model.predict(test)

In [ ]:
classes[np.argmax(predict)]

'cats'